**WSTĘP**  

Dataset pochodzi z badania SUPPORT. Celem I fazy badania było Opracowanie i walidacja modelu prognostycznego, który szacuje przeżycie w okresie 180 dni u ciężko chorych hospitalizowanych dorosłych, na podstawie szeregu informacji klinicznych, demograficznychi fizjologicznych.

W niniejszym projekcie koncentruję się na odtworzeniu tego zadania prognostycznego z użyciem współczesnych narzędzi uczenia maszynowego oraz przypisanego mi modelu SVM (Support Vector Machine). Projekt obejmuje pełny pipeline analityczny, od eksploracji danych i ich przygotowania, poprzez inżynierię cech i strojenie hiperparametrów, aż po końcową walidację modelu na wydzielonym zbiorze testowym.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

from sklearn.svm import SVC

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [18]:
df = pd.read_csv("support2.csv")

In [19]:
df.head()

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11.0,...,1.199951,141.0,7.459961,NaN,NaN,NaN,7.0,7.0,NaN,7.0
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,5.500000,132.0,7.250000,NaN,NaN,NaN,NaN,1.0,<2 mo. follow-up,1.0
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,2.000000,134.0,7.459961,NaN,NaN,NaN,1.0,0.0,<2 mo. follow-up,0.0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11.0,...,0.799927,139.0,NaN,NaN,NaN,NaN,0.0,0.0,no(M2 and SIP pres),0.0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NaN,...,0.799927,143.0,7.509766,NaN,NaN,NaN,NaN,2.0,no(M2 and SIP pres),2.0


In [4]:
df.tail()

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
9101,66.07300,0,male,0,23,350,ARF/MOSF w/Sepsis,ARF/MOSF,1,8.0,...,1.099854,131.0,7.459961,188.0,21.0,NaN,NaN,0.0,NaN,0.000000
9102,55.15399,0,female,0,29,347,Coma,Coma,1,11.0,...,5.899414,135.0,7.289062,190.0,49.0,0.0,NaN,0.0,NaN,0.000000
9103,70.38196,0,male,0,8,346,ARF/MOSF w/Sepsis,ARF/MOSF,1,NaN,...,2.699707,139.0,7.379883,189.0,60.0,3900.0,NaN,NaN,NaN,2.525391
9104,47.01999,1,male,1,7,7,MOSF w/Malig,ARF/MOSF,1,13.0,...,3.500000,135.0,7.469727,246.0,55.0,NaN,NaN,0.0,<2 mo. follow-up,0.000000
9105,81.53894,1,female,0,12,198,ARF/MOSF w/Sepsis,ARF/MOSF,1,8.0,...,1.199951,137.0,7.289062,187.0,15.0,NaN,0.0,NaN,no(M2 and SIP pres),0.494751


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9105 entries, 1 to 9105
Data columns (total 47 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       9105 non-null   float64
 1   death     9105 non-null   int64  
 2   sex       9105 non-null   object 
 3   hospdead  9105 non-null   int64  
 4   slos      9105 non-null   int64  
 5   d.time    9105 non-null   int64  
 6   dzgroup   9105 non-null   object 
 7   dzclass   9105 non-null   object 
 8   num.co    9105 non-null   int64  
 9   edu       7471 non-null   float64
 10  income    6123 non-null   object 
 11  scoma     9104 non-null   float64
 12  charges   8933 non-null   float64
 13  totcst    8217 non-null   float64
 14  totmcst   5630 non-null   float64
 15  avtisst   9023 non-null   float64
 16  race      9063 non-null   object 
 17  sps       9104 non-null   float64
 18  aps       9104 non-null   float64
 19  surv2m    9104 non-null   float64
 20  surv6m    9104 non-null   float64
 

In [ ]:
df.describe(include="all")

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
count,9105.000000,9105.000000,9105,9105.000000,9105.000000,9105.000000,9105,9105,9105.000000,7471.000000,...,9038.000000,9104.000000,6821.000000,4605.000000,4753.000000,4243.000000,3464.000000,6238.000000,7705,9105.000000
unique,NaN,NaN,2,NaN,NaN,NaN,8,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
top,NaN,NaN,male,NaN,NaN,NaN,ARF/MOSF w/Sepsis,ARF/MOSF,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<2 mo. follow-up,NaN
freq,NaN,NaN,5125,NaN,NaN,NaN,3515,4227,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3123,NaN
mean,62.650823,0.681054,NaN,0.259198,17.863042,478.449863,NaN,NaN,1.868644,11.747691,...,1.770961,137.568541,7.415364,159.873398,32.349463,2191.546047,1.157910,1.637384,NaN,1.888272
std,15.593710,0.466094,NaN,0.438219,22.006440,560.383272,NaN,NaN,1.344409,3.447743,...,1.686041,6.029326,0.080563,88.391541,26.792288,1455.245777,1.739672,2.231358,NaN,2.003763
min,18.041990,0.000000,NaN,0.000000,3.000000,3.000000,NaN,NaN,0.000000,0.000000,...,0.099991,110.000000,6.829102,0.000000,1.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,52.797000,0.000000,NaN,0.000000,6.000000,26.000000,NaN,NaN,1.000000,10.000000,...,0.899902,134.000000,7.379883,103.000000,14.000000,1165.500000,0.000000,0.000000,NaN,0.000000
50%,64.856990,1.000000,NaN,0.000000,11.000000,233.000000,NaN,NaN,2.000000,12.000000,...,1.199951,137.000000,7.419922,135.000000,23.000000,1968.000000,0.000000,1.000000,NaN,1.000000
75%,73.998960,1.000000,NaN,1.000000,20.000000,761.000000,NaN,NaN,3.000000,14.000000,...,1.899902,141.000000,7.469727,188.000000,42.000000,3000.000000,2.000000,3.000000,NaN,3.000000


In [20]:
df.columns

Index(['age', 'death', 'sex', 'hospdead', 'slos', 'd.time', 'dzgroup',
       'dzclass', 'num.co', 'edu', 'income', 'scoma', 'charges', 'totcst',
       'totmcst', 'avtisst', 'race', 'sps', 'aps', 'surv2m', 'surv6m', 'hday',
       'diabetes', 'dementia', 'ca', 'prg2m', 'prg6m', 'dnr', 'dnrday',
       'meanbp', 'wblc', 'hrt', 'resp', 'temp', 'pafi', 'alb', 'bili', 'crea',
       'sod', 'ph', 'glucose', 'bun', 'urine', 'adlp', 'adls', 'sfdm2',
       'adlsc'],
      dtype='object')

In [21]:
df.isna().sum().sort_values(ascending = False)

adlp        5641
urine       4862
glucose     4500
bun         4352
totmcst     3475
alb         3372
income      2982
adls        2867
bili        2601
pafi        2325
ph          2284
prg2m       1649
edu         1634
prg6m       1633
sfdm2       1400
totcst       888
wblc         212
charges      172
avtisst       82
crea          67
race          42
dnrday        30
dnr           30
hrt            1
meanbp         1
resp           1
sps            1
temp           1
sod            1
surv6m         1
surv2m         1
aps            1
scoma          1
age            0
dementia       0
ca             0
death          0
diabetes       0
hday           0
num.co         0
dzclass        0
dzgroup        0
d.time         0
slos           0
hospdead       0
sex            0
adlsc          0
dtype: int64

**CZYSZCZENIE DANYCH**  
Nasz target to kolumna death. Na początku w czyszczeniu danych zajmiemy się usunięciem kolumn powodujących "data leakage", które zawierają informacje powiązane z targetem, znane dopiero po czasie lub będące prognozami lekarzy. Później przejdziemy do czyszczenia danych mogących zaburzać predykcje ze względu na braki wartości.

In [22]:
df.head()

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,...,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11.0,...,1.199951,141.0,7.459961,NaN,NaN,NaN,7.0,7.0,NaN,7.0
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,5.500000,132.0,7.250000,NaN,NaN,NaN,NaN,1.0,<2 mo. follow-up,1.0
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,...,2.000000,134.0,7.459961,NaN,NaN,NaN,1.0,0.0,<2 mo. follow-up,0.0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11.0,...,0.799927,139.0,NaN,NaN,NaN,NaN,0.0,0.0,no(M2 and SIP pres),0.0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NaN,...,0.799927,143.0,7.509766,NaN,NaN,NaN,NaN,2.0,no(M2 and SIP pres),2.0


In [23]:
drop_cols = ["hospdead", "surv2m", "surv6m", "prg2m", "prg6m", "d.time", "dnr", "dnrday"]
df = df.drop(columns = drop_cols)

In [29]:
missing = df.isna().sum().sort_values(ascending = False)
n_rows = len(df)
missing_p = (missing/n_rows*100).round(1)
pd.DataFrame({
    "missing" : missing,
    "missing percentage": missing_p
})

,missing,missing percentage
adlp,5641,62.0
urine,4862,53.4
glucose,4500,49.4
bun,4352,47.8
totmcst,3475,38.2
alb,3372,37.0
income,2982,32.8
adls,2867,31.5
bili,2601,28.6
pafi,2325,25.5


In [32]:
# usuwamy kolumny, w których braki danych były większe niż 40 procent.
high_missing_cols = ["adlp", "urine", "glucose", "bun"]
df = df.drop(columns = high_missing_cols)

In [33]:
df.head()

,age,death,sex,slos,dzgroup,dzclass,num.co,edu,income,scoma,...,temp,pafi,alb,bili,crea,sod,ph,adls,sfdm2,adlsc
1,62.84998,0,male,5,Lung Cancer,Cancer,0,11.0,$11-$25k,0.0,...,36.00000,388.00000,1.799805,0.199982,1.199951,141.0,7.459961,7.0,NaN,7.0
2,60.33899,1,female,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,$11-$25k,44.0,...,34.59375,98.00000,NaN,NaN,5.500000,132.0,7.250000,1.0,<2 mo. follow-up,1.0
3,52.74698,1,female,17,Cirrhosis,COPD/CHF/Cirrhosis,2,12.0,under $11k,0.0,...,37.39844,231.65625,NaN,2.199707,2.000000,134.0,7.459961,0.0,<2 mo. follow-up,0.0
4,42.38498,1,female,3,Lung Cancer,Cancer,2,11.0,under $11k,0.0,...,35.00000,NaN,NaN,NaN,0.799927,139.0,NaN,0.0,no(M2 and SIP pres),0.0
5,79.88495,0,female,16,ARF/MOSF w/Sepsis,ARF/MOSF,1,NaN,NaN,26.0,...,37.89844,173.31250,NaN,NaN,0.799927,143.0,7.509766,2.0,no(M2 and SIP pres),2.0


In [37]:
numeric_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = df.select_dtypes(include = ["object", "category"]).columns.tolist()
numeric_features = [col for col in numeric_features if col != "death"]

In [38]:
numeric_features

['age',
 'slos',
 'num.co',
 'edu',
 'scoma',
 'charges',
 'totcst',
 'totmcst',
 'avtisst',
 'sps',
 'aps',
 'hday',
 'diabetes',
 'dementia',
 'meanbp',
 'wblc',
 'hrt',
 'resp',
 'temp',
 'pafi',
 'alb',
 'bili',
 'crea',
 'sod',
 'ph',
 'adls',
 'adlsc']

In [36]:
categorical_features

['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca', 'sfdm2']

**PREPROCESSING I MODEL**  
Dzielimy dane na X i y oraz train i test, zamieniamy braki danych medianą(dla danych numerycznych) oraz najczęstszą kategorią(dla danych kategorycznych)

In [41]:
target = "death"
X = df.drop(columns = [target])
y = df[target]

In [42]:
print(y.value_counts())

death
1    6201
0    2904
Name: count, dtype: int64


In [43]:
print(y.value_counts(normalize = True))

death
1    0.681054
0    0.318946
Name: proportion, dtype: float64


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state=35)
# dzielimy dane tak, żeby ich rozkład był podobny do tego na całym zbiorze (stratify)
# oraz dla powtarzalności wyników ustawiamy random_state

In [ ]:
print("Cały zbiór:")
print(y.value_counts(normalize=True))

print("\nTrain:")
print(y_train.value_counts(normalize=True))

print("\nTest:")
print(y_test.value_counts(normalize=True))


Cały zbiór:
death
1    0.681054
0    0.318946
Name: proportion, dtype: float64

Train:
death
1    0.681082
0    0.318918
Name: proportion, dtype: float64

Test:
death
1    0.680945
0    0.319055
Name: proportion, dtype: float64


In [48]:
numeric_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown = "ignore"))
])
preprocessor = ColumnTransformer(
    transformers = [
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [49]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'slos', 'num.co', 'edu', 'scoma',
                                  'charges', 'totcst', 'totmcst', 'avtisst',
                                  'sps', 'aps', 'hday', 'diabetes', 'dementia',
                                  'meanbp', 'wblc', 'hrt', 'resp', 'temp',
                                  'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
                                  'adls', 'adlsc']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['sex', 'dzgroup', 'dzclass', 'income', 'race',
                                  'ca', 'sfdm2'])])

In [51]:
clf = Pipeline(steps = [
    ("preprocess", preprocessor),
    ("model", SVC(
        probability = True,
        class_weight = "balanced",
        random_state = 35
    ))
])

In [52]:
clf

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'slos', 'num.co',
                                                   'edu', 'scoma', 'charges',
                                                   'totcst', 'totmcst',
                                                   'avtisst', 'sps', 'aps',
                                                   'hday', 'diabetes',
                                                   'dementia', 'meanbp', 'wblc',
                                                   'hrt', 'resp', 'temp',
                                                   'pafi', 'alb', 'bili',
                                                   'crea', 'sod', 'ph', 'adls',
                                                   'adlsc']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'dzgroup', 'dzclass',
                                                   'income', 'race', 'ca',
                                                   'sfdm2'])])),
                ('model',
                 SVC(class_weight='balanced', probability=True,
                     random_state=35))])

**GridSearchCV**

In [ ]:
# sprawdzamy parę możliwych kombinacji modelu, żeby wybrać ten najlepszy
param_grid = {
    "model__C": [0.1, 1],         # mała siatka na start
    "model__kernel": ["linear", "rbf"],  # tylko kernel liniowy
}
# robimy 4-krotną walidację krzyżową
grid_search = GridSearchCV(
    clf,
    param_grid,
    cv = 4,         
    scoring = "f1",
    n_jobs = -1,
    verbose = 1
)

grid_search.fit(X_train, y_train)

print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepszy wynik CV (F1):", grid_search.best_score_)

# UWAGA
# Ze względu na ograniczenie obliczeniowe zastosowałam uproszczoną siatkę hiperparametrów. W praktycznych zastosowaniach można rozważyć gęstszą siatkę, lub więcej parametrów C.


Fitting 4 folds for each of 4 candidates, totalling 16 fits
Najlepsze parametry: {'model__C': 0.1, 'model__kernel': 'rbf'}
Najlepszy wynik CV (F1): 0.8076369793834542


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_probab = best_model.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test, y_pred)  # daje nam procent poprawnych predykcji
f1 = f1_score(y_test, y_pred)  # czy model skutecznie wykrywa zgony
roc = roc_auc_score(y_test, y_probab) # jak dobrze model odróżnia klasę 1 od 0 na podstawie prawdpopodobieństwa

print("Wyniki na zbiorze testowym:")
print("Accuracy:", acc)
print("F1-score:", f1)
print("ROC-AUC:", roc)

print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

Wyniki na zbiorze testowym:
Accuracy: 0.756727073036793
F1-score: 0.8054457619675011
ROC-AUC: 0.842947365498862

Classification report:
               precision    recall  f1-score   support

           0       0.59      0.79      0.68       581
           1       0.88      0.74      0.81      1240

    accuracy                           0.76      1821
   macro avg       0.74      0.77      0.74      1821
weighted avg       0.79      0.76      0.76      1821


Confusion matrix:
 [[461 120]
 [323 917]]


**Interpretacja wyników modelu SVM na zbiorze testowym**   

Na zbiorze testowym najlepszy model (SVC z jądrem RBF i C=0.1) osiągnął:
- Accuracy: 0.7567
- F1-score: 0.8054
- ROC-AUC: 0.8429

Wyniki te wskazują, że model ogólnie poprawnie klasyfikuje około 76% przypadków, a jego zdolność do wykrywania pacjentów zagrożonych zgonem jest wysoka (F1 > 0.80). Wysoka wartość ROC-AUC (0.84) oznacza dobrą separację pomiędzy pacjentami, którzy przeżyją, a tymi, którzy umrą.

Analiza classification report pokazuje, że:
- *Dla klasy 1* (zgony):
 Model uzyskał precision = 0.88 i recall = 0.74, co oznacza, że potrafi skutecznie identyfikować pacjentów w stanie krytycznym.
- *Dla klasy 0* (przeżycie)R
Recall wynosi 0.79, co oznacza, że większość pacjentów, którzy przeżywają, jest poprawnie klasyfikowana.

Confusion matrix potwierdza, że model:
- poprawnie wykrył 917 przypadków zgonów,
- pomylił 323 zgony jako przeżycia,
- wygenerował 120 fałszywych zgonów,
- poprawnie wskazał 461 przypadków przeżycia.

Podsumowując:
Model dobrze sprawdza się jako narzędzie wspierające ocenę ryzyka 180-dniowego zgonu, z dobrą jakością predykcji i stabilnym rozróżnianiem klas.